In [1]:
from functions import *
from ipywidgets import HBox, HTML, Dropdown, Label, Layout,Image
import tkinter as tk
from IPython.display import display, Markdown, clear_output, IFrame,Javascript
from pyvis.network import Network
pathways_name=pd.read_csv("data/pathways.tsv", sep='\t')["pathway_name"]
last_selection=pathways_name
display(HTML("<style>.container { width:100% !important; }</style>"))


HTML(value='<style>.container { width:100% !important; }</style>')

In [2]:
display(HTML("<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>"))
text=Label(value="")
text.add_class("left-spacing-class")
menu = Dropdown(
       options=list(pathways_name.values),
       value=list(pathways_name.values)[25],
       description='Pathway:')
sidebyside=HBox([menu, HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells([2])"><input type="submit" id="toggleButton" value="Run"></form>'''),text])

display(sidebyside)

HTML(value='<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>')

In [5]:
skip_calcs=False
selection=menu.value
pathway_edges=read_pathway(selection)
if (len(pathway_edges)==0):
    selection=last_selection
    skip_calcs=True
    st.error("Edges not found, try another pathway!")
else:
    skip_calcs=False
    last_selection=selection
    pathway_edges=read_pathway(selection)
    adj_matrix,nodes_renamed,inv_nodes_renamed=build_adj(pathway_edges)
    G = nx.from_numpy_matrix(adj_matrix)
    triad_cliques=get_triad(G)
    weighted_edges=calculate_weighted_edges(triad_cliques, adj_matrix,inv_nodes_renamed)
    to_remove=[]
    signify_values={}
    essential_edges=[]
    for x in weighted_edges.items():
        zeros=0
        ones=0
        minus=0
        for z in x[1]:
            if (z[1]==0):
                zeros+=1
            elif (z[1]==1):
                ones+=1
            else:
                minus+=1
        if (ones==0):
            if (minus==0):
                to_remove.append(x[0])
            else:
                m=(zeros+minus)/2
                if ((minus+zeros)/(zeros*minus+1)>=((minus+zeros)/((m+1)*(m-1)+1))):
                    to_remove.append(x[0])
        else:
            essential_edges.append(x[0])
        if (ones==0):
            signify_values[x[0]]=round((minus+zeros)/(zeros*minus+1),2)
        else:
            signify_values[x[0]]=0

    relabel={}
    for e,node in enumerate( G.nodes()):
        relabel[e]=str(inv_nodes_renamed[node])
    net=Network(height="825px",notebook=True,directed=True,width="1800px", bgcolor='#222222', font_color='white')
    for i,node in relabel.items():
        net.add_node(str(node))

    for edge in pathway_edges.values:
            if(edge[2]==-1):
                net.add_edge(str(edge[0]), str(edge[1]), color="yellow")
            else:
                net.add_edge(str(edge[0]), str(edge[1]))
    for triad in triad_cliques:
        for i,x in enumerate(triad):
            for j,y in enumerate(triad):
                value=""
                isessential=""
                weight=pathway_edges[(pathway_edges[0]==inv_nodes_renamed[triad[i]]) & (pathway_edges[1]==inv_nodes_renamed[triad[j]])]
                if (weight.empty):
                    continue
                if ((str(inv_nodes_renamed[triad[i]])+","+str(inv_nodes_renamed[triad[j]])) in to_remove) or ((str(inv_nodes_renamed[triad[j]])+","+str(inv_nodes_renamed[triad[i]])) in to_remove):                    
                    color="red"
                    size=10
                    value+=", significativity:  "+str(signify_values[str(inv_nodes_renamed[triad[j]])+","+str(inv_nodes_renamed[triad[i]])])
                else:
                    color="green"
                    size=3
                    value+=", significativity:  "+str(signify_values[str(inv_nodes_renamed[triad[j]])+","+str(inv_nodes_renamed[triad[i]])])
                if ((str(inv_nodes_renamed[triad[i]])+","+str(inv_nodes_renamed[triad[j]])) in essential_edges) or ((str(inv_nodes_renamed[triad[j]])+","+str(inv_nodes_renamed[triad[i]])) in essential_edges):
                    isessential="Essential "
                weight=int(weight[2].values)
                if (weight==1):
                    net.add_edge(str(inv_nodes_renamed[triad[i]]), str(inv_nodes_renamed[triad[j]]), color=color, width=size,title=isessential+"Expression edge"+value)
                else:
                    net.add_edge(str(inv_nodes_renamed[triad[i]]), str(inv_nodes_renamed[triad[j]]), color=color, width=size,title=+isessential+"Suppression edge"+value)
    net.hrepulsion(node_distance=120, central_gravity=0.0, spring_length=100, spring_strength=0, damping=0.09)
    net.show("data/graph.html")

KeyError: '51129,6256'

In [6]:
signify_values

{'6256,4023': 0,
 '51129,4023': 0.67,
 '6256,51129': 1.0,
 '6258,4023': 1.0,
 '6258,51129': 1.0,
 '6257,4023': 1.0,
 '6257,51129': 1.0,
 '5465,4023': 1.0,
 '5465,51129': 1.0,
 '5467,4023': 1.0,
 '5467,51129': 1.0,
 '5468,4023': 0,
 '5468,51129': 1.0}

In [7]:
weighted_edges

{'6256,4023': [(0.751351887950401, 1)],
 '51129,4023': [(0.00958993913913588, 0),
  (0.42793669112729465, -1),
  (0.493972998423884, -1),
  (0.4154314035435193, -1),
  (0.47398475209831153, -1),
  (-0.08148364214020982, 0)],
 '6256,51129': [(0.00958993913913588, 0)],
 '6258,4023': [(0.31271902702610976, -1)],
 '6258,51129': [(0.42793669112729465, -1)],
 '6257,4023': [(-0.18965576363770847, -1)],
 '6257,51129': [(0.493972998423884, -1)],
 '5465,4023': [(0.5002211698648119, -1)],
 '5465,51129': [(0.4154314035435193, -1)],
 '5467,4023': [(-0.11442379562472764, -1)],
 '5467,51129': [(0.47398475209831153, -1)],
 '5468,4023': [(0.99691599298314, 1)],
 '5468,51129': [(-0.08148364214020982, 0)]}